In [ ]:
import subprocess
from datetime import datetime
from pathlib import Path
from typing import Literal

import fsspec
from utils import sync_notebook

In [ ]:
fs = fsspec.filesystem("file")
current_date = datetime.today().strftime("%Y-%m-%d")
output_folder = f"results/{current_date}/"
files = fs.glob(f"{output_folder}pyinstrument-*.json")
fs.rm(files)
fs.mkdirs(output_folder, exist_ok=True)

In [ ]:
dataset = "gpm_imerg"

In [ ]:
def run_pyinstrument(file: str, output_format: Literal["json", "html"]):
    output_file = (
        output_folder
        + "pyinstrument-"
        + dataset
        + "-"
        + Path(file).stem
        + "."
        + output_format
    )
    command = [
        "pyinstrument",
        "--outfile",
        output_file,
        "-r",
        output_format,
        file,
        "--dataset",
        dataset,
    ]
    subprocess.run(command)

In [ ]:
input_methods = ["resample-rioxarray-h5netcdf", "resample-odc", "resample-xesmf"]

In [ ]:
notebooks = []
for fp in input_methods:
    notebooks.extend(fs.glob(f"{fp}*.ipynb"))

In [ ]:
for file in notebooks:
    sync_notebook(file)

In [ ]:
modules = []
for fp in input_methods:
    modules.extend(fs.glob(f"{fp}*.py"))
for file in modules:
    run_pyinstrument(file, "json")